# Fragile Families Project

In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn_pandas import DataFrameMapper

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
######## VARIABLE OF INTEREST ########
test_var = 'materialHardship'

## Preprocessing

Data was first imputed naively by removing columns that were completely NA, then filling in missing data with modes

In [3]:
features = pd.read_csv('background_imputed_3.csv', low_memory=False, index_col='challengeID')
print("Features shape: {}".format(features.shape))
features.sort_index(kind='mergesort', inplace=True) # mergesort is the only stable sort
features.head()

Features shape: (4242, 12007)


,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1.0,1854.0,18540.0,1.0,0.0,40.0,0.0,1.0,202.4850,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,7986.0,189.0,0.0,0.0
2.0,4470.0,44700.0,1.0,0.0,40.0,1.0,1.0,45.6082,1.0,1.0,...,12.1322,10.99190,9.84507,1.0,10.97270,9.72355,3280.0,2552.0,24.0,15.0
3.0,371.0,3710.0,1.0,0.0,35.0,1.0,1.0,39.0603,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,4597.0,1925.0,45.0,0.0
4.0,2654.0,26540.0,1.0,0.0,30.0,1.0,1.0,22.3049,1.0,1.0,...,12.1322,10.67730,10.56410,1.0,8.52233,10.60810,263.0,4494.0,12.0,1.0
5.0,2810.0,28100.0,1.0,0.0,25.0,1.0,1.0,35.5183,1.0,1.0,...,12.1322,9.73198,10.56410,1.0,10.11530,9.64647,1005.0,4067.0,29.0,0.0


In [4]:
# holders for later on...

n = len(features)+1
pred_gpa_y = np.full(n-1, 2.866)
pred_grit_y = np.full(n-1, 3.427)
pred_material_y = np.full(n-1, 0.0596)
pred_evict_y = np.full(n-1, 0.0596)
pred_layoff_y = np.full(n-1, 0.209)
pred_job_y = np.full(n-1, 0.334)

In [5]:
# some columns are not numeric; encode them
from sklearn.preprocessing import LabelEncoder
str_col_features = features.select_dtypes(include=['object'])
label_encoders={} # save the label encoders to allow for inverse transforms
for col_name in str_col_features.columns:
    le = LabelEncoder()
    dmy = le.fit(features[col_name])
    features[col_name] = le.transform(features[col_name])
    label_encoders[col_name] = le
features.head()

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1.0,1854.0,18540.0,1.0,0.0,40.0,0.0,1.0,202.4850,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,7986.0,189.0,0.0,0.0
2.0,4470.0,44700.0,1.0,0.0,40.0,1.0,1.0,45.6082,1.0,1.0,...,12.1322,10.99190,9.84507,1.0,10.97270,9.72355,3280.0,2552.0,24.0,15.0
3.0,371.0,3710.0,1.0,0.0,35.0,1.0,1.0,39.0603,1.0,1.0,...,12.1322,10.99190,10.56410,1.0,10.97270,9.56668,4597.0,1925.0,45.0,0.0
4.0,2654.0,26540.0,1.0,0.0,30.0,1.0,1.0,22.3049,1.0,1.0,...,12.1322,10.67730,10.56410,1.0,8.52233,10.60810,263.0,4494.0,12.0,1.0
5.0,2810.0,28100.0,1.0,0.0,25.0,1.0,1.0,35.5183,1.0,1.0,...,12.1322,9.73198,10.56410,1.0,10.11530,9.64647,1005.0,4067.0,29.0,0.0


In [6]:
# scale data to unit variance and 0 mean
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
indices_f = features.index
colnames_f = features.columns
features = ss.fit_transform(features)
features = pd.DataFrame(data=features, index=indices_f, columns=colnames_f)
features.head()

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
1.0,-0.416725,-0.416725,0.0,-0.30666,0.385580,-1.880664,0.162398,0.702597,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,-0.586254,2.694875,-2.617362,-1.333624,-0.548459
2.0,1.439058,1.439058,0.0,-0.30666,0.385580,0.531727,0.162398,-0.184623,0.601133,0.162398,...,0.340443,0.392404,-1.256213,0.0,0.401826,-0.273756,0.515323,-0.611222,-0.602345,0.425379
3.0,-1.468761,-1.468761,0.0,-0.30666,-0.051404,0.531727,0.162398,-0.221655,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,-0.586254,1.125283,-1.143532,0.037524,-0.548459
4.0,0.150793,0.150793,0.0,-0.30666,-0.488388,0.531727,0.162398,-0.316416,0.601133,0.162398,...,0.340443,-0.040350,0.307392,0.0,-3.309481,1.488338,-0.881979,1.037498,-0.967985,-0.483537
5.0,0.261459,0.261459,0.0,-0.30666,-0.925373,0.531727,0.162398,-0.241687,0.601133,0.162398,...,0.340443,-1.340702,0.307392,0.0,-0.896784,-0.427306,-0.538327,0.674983,-0.449996,-0.548459


In [7]:
from sklearn.model_selection import train_test_split

full_y = pd.read_csv('train.csv', index_col='challengeID')
train_y = full_y[test_var] # variable we are interested in
print("Target shape: {}".format(train_y.shape))
train_y.dropna(how='any', inplace=True) # drop those with no reported outcome
print("Target shape (no na): {}".format(train_y.shape))

# get the rows for which we can predict
y_indices = train_y.index.values.tolist()
full_x = features
train_no_na_x = features.loc[y_indices]
print("Features shape (final): {}".format(train_no_na_x.shape))
train_x = train_no_na_x

# train_x.to_csv("full_x.csv", index=True, header="challengeID")
# train_y.to_csv("full_y.csv", index=True, header="challengeID")

train_x, test_x, train_y, test_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=0)

train_x.shape
train_x.head()
train_y.shape
train_y.head()
test_x.shape
test_x.head()
test_y.shape
test_y.head()

Target shape: (2121,)
Target shape (no na): (1459,)
Features shape (final): (1459, 12007)


(1167, 12007)

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
266,-1.055183,-1.055183,0.0,-0.30666,1.783930,-1.880664,0.162398,0.023866,-1.663526,0.162398,...,0.340443,-2.541036,0.307392,0.0,-3.466408,-0.586254,-0.855117,1.206445,-0.724225,-0.094001
2420,0.899207,0.899207,0.0,-0.30666,-0.488388,0.531727,0.162398,4.557555,0.601133,0.162398,...,0.340443,-0.134851,0.307392,0.0,-0.836352,2.145127,-0.881979,1.153808,-0.876575,-0.483537
1004,-0.606844,-0.606844,0.0,-0.30666,-0.051404,0.531727,0.162398,-0.092858,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,-0.586254,-0.603630,0.283604,-0.297646,-0.288769
1875,1.617826,1.617826,0.0,-0.30666,-0.051404,-1.880664,0.162398,0.054707,0.601133,0.162398,...,0.340443,0.392404,-3.878612,0.0,0.401826,0.786528,1.067853,-1.380398,-0.236706,-0.353692
104,-1.496428,-1.496428,0.0,-0.30666,1.696534,0.531727,0.162398,-0.079920,0.601133,0.162398,...,-0.280441,0.392404,0.307392,0.0,0.401826,2.213256,-0.142340,0.034852,1.530551,0.165689


(1167,)

challengeID
266     0.000000
2420    0.000000
1004    0.000000
1875    0.181818
104     0.181818
Name: materialHardship, dtype: float64

(292, 12007)

,# idnum,mothid1,m1intmon,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,...,f4d9,m5c6,m5d20,m5k10,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
4082,0.543799,0.543799,0.0,-0.306660,-0.226198,0.531727,0.162398,-0.382195,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,0.175358,-0.876885,0.674983,-0.785165,-0.353692
1403,0.521807,0.521807,0.0,-0.306660,-0.051404,0.531727,0.162398,-0.409558,0.601133,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,0.786528,0.602394,-0.939777,0.494573,0.425379
4056,0.672909,0.672909,0.0,-0.306660,-0.051404,0.531727,0.162398,-0.108313,-1.663526,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,3.923856,-0.225706,-0.033066,-0.297646,-0.483537
1608,-1.476565,-1.476565,0.0,-0.306660,-0.488388,0.531727,0.162398,-0.397717,0.601133,0.162398,...,-3.038644,0.392404,0.307392,0.0,0.401826,-0.377145,-0.882443,1.023914,0.494573,0.035844
1383,-1.550342,-1.550342,0.0,2.490869,-0.488388,-1.880664,0.162398,-0.407921,-1.663526,0.162398,...,0.340443,0.392404,0.307392,0.0,0.401826,0.786528,0.283752,-0.554340,0.037524,0.944759


(292,)

challengeID
4082    0.000000
1403    0.454545
4056    0.090909
1608    0.000000
1383    0.363636
Name: materialHardship, dtype: float64

## Feature Selection

First remove features with 0 variance so that real feature selection can be sped up

In [8]:
from sklearn.feature_selection import VarianceThreshold
# remove features with 0 variance

# mapper from pandas to numpy
mapper = DataFrameMapper([(train_x.columns, None)])
indices = train_x.index
colnames = train_x.columns

vt = VarianceThreshold()
train_vt_x = vt.fit_transform(mapper.fit_transform(train_x))
train_x = pd.DataFrame(data=train_vt_x, index=indices, columns=colnames[vt.get_support(True)])
train_x.head()

# Test set needs variance threshold treatment
# mapper from pandas to numpy
mapper_test = DataFrameMapper([(test_x.columns, None)])
indices = test_x.index
colnames = test_x.columns

test_vt_x = vt.transform(mapper.fit_transform(test_x))
test_x = pd.DataFrame(data=test_vt_x, index=indices, columns=colnames[vt.get_support(True)])
test_x.head()

,# idnum,mothid1,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,m1a3,...,f4d7,f4d9,m5c6,m5d20,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
266,-1.055183,-1.055183,-0.30666,1.783930,-1.880664,0.162398,0.023866,-1.663526,0.162398,-0.312772,...,2.707464,0.340443,-2.541036,0.307392,-3.466408,-0.586254,-0.855117,1.206445,-0.724225,-0.094001
2420,0.899207,0.899207,-0.30666,-0.488388,0.531727,0.162398,4.557555,0.601133,0.162398,-0.312772,...,1.629972,0.340443,-0.134851,0.307392,-0.836352,2.145127,-0.881979,1.153808,-0.876575,-0.483537
1004,-0.606844,-0.606844,-0.30666,-0.051404,0.531727,0.162398,-0.092858,0.601133,0.162398,-0.312772,...,-0.472812,0.340443,0.392404,0.307392,0.401826,-0.586254,-0.603630,0.283604,-0.297646,-0.288769
1875,1.617826,1.617826,-0.30666,-0.051404,-1.880664,0.162398,0.054707,0.601133,0.162398,-0.312772,...,-0.472812,0.340443,0.392404,-3.878612,0.401826,0.786528,1.067853,-1.380398,-0.236706,-0.353692
104,-1.496428,-1.496428,-0.30666,1.696534,0.531727,0.162398,-0.079920,0.601133,0.162398,-0.312772,...,-0.472812,-0.280441,0.392404,0.307392,0.401826,2.213256,-0.142340,0.034852,1.530551,0.165689


,# idnum,mothid1,m1lenhr,m1lenmin,cm1fint,cm1citsm,m1citywt,innatsm,incitysm,m1a3,...,f4d7,f4d9,m5c6,m5d20,f5c6,k5f1,-9,-6,-2,-1
challengeID,,,,,,,,,,,,,,,,,,,,,
4082,0.543799,0.543799,-0.306660,-0.226198,0.531727,0.162398,-0.382195,0.601133,0.162398,-0.312772,...,-0.182628,0.340443,0.392404,0.307392,0.401826,0.175358,-0.876885,0.674983,-0.785165,-0.353692
1403,0.521807,0.521807,-0.306660,-0.051404,0.531727,0.162398,-0.409558,0.601133,0.162398,-0.312772,...,-0.472812,0.340443,0.392404,0.307392,0.401826,0.786528,0.602394,-0.939777,0.494573,0.425379
4056,0.672909,0.672909,-0.306660,-0.051404,0.531727,0.162398,-0.108313,-1.663526,0.162398,-0.312772,...,-0.472812,0.340443,0.392404,0.307392,0.401826,3.923856,-0.225706,-0.033066,-0.297646,-0.483537
1608,-1.476565,-1.476565,-0.306660,-0.488388,0.531727,0.162398,-0.397717,0.601133,0.162398,-0.312772,...,-0.472812,-3.038644,0.392404,0.307392,0.401826,-0.377145,-0.882443,1.023914,0.494573,0.035844
1383,-1.550342,-1.550342,2.490869,-0.488388,-1.880664,0.162398,-0.407921,-1.663526,0.162398,-0.312772,...,-0.472812,0.340443,0.392404,0.307392,0.401826,0.786528,0.283752,-0.554340,0.037524,0.944759


Next use some regression model to find most relevant features.  Here we select features based on Lasso and mutual info as well as importance assigned to features based on decision trees.

In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel

from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# mapper from pandas to numpy
mapper = DataFrameMapper([(train_x.columns, None)])
indices = train_x.index
colnames = train_x.columns

In [17]:
# Use Lasso Regression
if not test_var == 'layoff':
    fs_model_lasso = LassoCV().fit(mapper.fit_transform(train_x), train_y.as_matrix())
    fs_lasso = SelectFromModel(fs_model_lasso, prefit=True, threshold=0.001)
    train_fs_lasso_x = fs_lasso.transform(train_x)
    train_fs_lasso_x = pd.DataFrame(data=train_fs_lasso_x, index=indices, columns=colnames[fs_lasso.get_support(True)])
    train_fs_lasso_x.head()

    scores_lasso = np.column_stack((colnames, fs_model_lasso.coef_))
    scores_lasso = scores_lasso[fs_lasso.get_support(True)]
    scores_lasso = np.core.records.fromarrays(scores_lasso.transpose(), names='name, score', formats = 'S8, f8')
    scores_lasso = np.sort(scores_lasso, order=['score'], kind='mergesort')
    print("Lasso # of features: {}".format(len(scores_lasso)))
    print(scores_lasso['name'])

    np.savetxt("{}_features_lasso.csv".format(test_var), scores_lasso['name'], fmt="%s")

    # fig1 = plt.figure(1)
    # dmy = fig1.clear()
    # y_pos1 = np.arange(len(scores_lasso))
    # dmy = plt.title('Lasso')
    # dmy = plt.bar(y_pos1, scores_lasso['score'], align='center')
    # dmy = plt.xticks(y_pos1, scores_lasso['name'], rotation='vertical')
    # fig1

,m2h9a1,f2a3d,m3b24,m3i7f,m3i16a_2,m3i23d,m4b2,m4b19,m4e20b,m4i8c1,...,p5m3c,p5q2d,p5q3ag,p5q3bl,hv3d2,hv3j7,hv3r10a5,hv4f1f,kind_a2,ffcc_centsurvey_c9_7
challengeID,,,,,,,,,,,,,,,,,,,,,
266,0.527348,-0.046605,0.257614,0.264313,0.192779,0.494690,0.807274,0.082967,-0.145621,-0.364728,...,-0.179746,0.161788,3.709221,2.840525,-0.509780,1.012575,-0.073834,-0.746431,0.108102,-0.015356
2420,0.527348,-0.046605,0.257614,-3.783395,0.192779,0.494690,-0.601968,0.082967,-0.145621,-0.364728,...,1.351347,0.161788,-0.222442,-0.276264,-0.509780,-0.377804,-0.073834,0.257488,0.108102,-0.015356
1004,0.527348,-0.046605,0.257614,0.264313,0.192779,-2.021468,-0.601968,0.082967,-0.145621,-0.364728,...,-0.179746,0.161788,-0.222442,-0.276264,-0.509780,-0.841263,-0.073834,1.261407,0.108102,-0.015356
1875,-1.896281,-0.046605,0.257614,0.264313,0.192779,0.494690,-0.601968,0.082967,-0.145621,1.650474,...,-0.179746,0.161788,-0.222442,-0.276264,-0.509780,-0.841263,-0.073834,1.261407,0.108102,-0.015356
104,-1.896281,-0.046605,0.257614,0.264313,0.192779,0.494690,-0.601968,0.082967,-0.145621,-0.364728,...,-0.179746,0.161788,-0.222442,-0.276264,1.961632,-0.377804,-0.073834,1.261407,0.108102,-0.015356


Lasso # of features: 42
['m5f23e' 'm4i23n' 'm5f23a' 'm4i23d' 'm5f23k' 'm3i23d' 'm5f23g' 'f5i19a'
 'm2h9a1' 'm3i7f' 'm3i16a_2' 'cm4hhinc' 'hv4f1f' 'm5f23c' 'p5q2d' 'm4b19'
 'kind_a2' 'm3b24' 'n5a6_2' 'm4i23k' 'p5m3c' 'f4e7a2' 'p5q3bl' 'f4c21b'
 'p5j11' 'm4i8c1' 'hv3j7' 'f5g2d' 'f5a6g02_' 'p5q3ag' 'm4b2' 'f5a6g03_'
 'm4j24b_2' 'hv3r10a5' 'm5i3a_8' 'm4e20b' 'f5i14b4' 'm5g18' 'f2a3d' 'hv3d2'
 'ffcc_cen' 'm5g0']


In [11]:
# new_sel_x = fs_lasso.transform(vt.transform(train_no_na_x))
# new_sel_x = pd.DataFrame(data=new_sel_x, index=y_indices, columns=colnames[fs_lasso.get_support(True)] )
# new_sel_x.head()
# new_sel_x.loc[[1875]]
# train_fs_lasso_x.loc[[7]]
# new_sel_x.to_csv("new_sel_x.csv", index=True, header="challengeID")

In [12]:
# Decision Tree to select features
# if test_var == 'gpa' or test_var == 'materialHardship': # regressor
fs_model_decision = DecisionTreeRegressor().fit(mapper.fit_transform(train_x), train_y.as_matrix())
# else: # classifier
#     fs_model_decision = DecisionTreeClassifier().fit(mapper.fit_transform(train_x), train_y.as_matrix())
    
fs_decision = SelectFromModel(fs_model_decision, prefit=True, threshold=0.01)
train_fs_decision_x = fs_decision.transform(train_x)
train_fs_decision_x = pd.DataFrame(data=train_fs_decision_x, index=indices, columns=colnames[fs_decision.get_support(True)])
train_fs_decision_x.head()

scores_decision = np.column_stack((colnames, fs_model_decision.feature_importances_))
scores_decision = scores_decision[fs_decision.get_support(True)]
scores_decision = np.core.records.fromarrays(scores_decision.transpose(), names='name, score', formats = 'S8, f8')
scores_decision = np.sort(scores_decision, order=['score'], kind='mergesort')
print("Decision # of features: {}".format(len(scores_decision)))
print(scores_decision['name'])

np.savetxt("{}_features_decision.csv".format(test_var), scores_decision['name'], fmt="%s")
        

# fig2 = plt.figure(2)
# dmy = fig2.clear()
# y_pos2 = np.arange(len(scores_decision))
# dmy = plt.title('Decision Tree')
# dmy = plt.bar(y_pos2, scores_decision['score'], align='center')
# dmy = plt.xticks(y_pos2, scores_decision['name'], rotation='vertical')
# fig2

,m2h8b,m3f3a,m3i0p,m4b3b_2,m4e21f,m4i8c2,cm4hhinc,cm4povco,f4b4a7,f4b8f,...,p5j2h,p5l18,p5q3dj,cm5b_ageyrs,hv3g1e,hv3l1,hv3p7_c,hv3mwtkg,hv3waz,kind_b1e
challengeID,,,,,,,,,,,,,,,,,,,,,
266,0.368433,-0.155675,-0.311728,0.239433,-0.186557,-0.629103,-0.249274,-0.239186,-1.106464,-0.174276,...,-1.293803,-0.561654,-3.219899,-0.452159,-1.013071,-0.484774,-0.158534,-1.003938,-1.343664,0.100413
2420,0.368433,-0.155675,-0.311728,0.239433,-0.186557,-0.629103,0.109504,-0.409103,-1.106464,-0.174276,...,3.899529,-0.561654,0.557950,-0.584887,0.110738,-0.484774,-0.158534,1.000034,0.095394,1.846119
1004,0.368433,-0.155675,-0.311728,-4.176539,-0.186557,-0.629103,2.206003,1.892147,-0.287476,-0.174276,...,-1.293803,1.680734,0.557950,0.476940,1.234548,-0.484774,-0.158534,0.191203,0.095394,0.100413
1875,0.368433,-0.155675,-0.311728,0.239433,-0.186557,0.714685,-0.746030,-0.434184,-0.287476,-0.174276,...,-1.293803,-0.561654,0.557950,1.903771,0.110738,-0.484774,-0.158534,-0.399062,0.095394,0.100413
104,0.368433,-0.155675,-0.311728,0.239433,-0.186557,1.916454,-0.522543,-0.125211,-0.287476,-0.174276,...,-1.293803,2.801928,0.557950,2.268774,0.110738,-0.484774,-0.158534,2.390589,0.095394,0.100413


Decision # of features: 27
['f4i23p1' 'm3f3a' 'f4b4a7' 'm5j9b1' 'cm5b_age' 'p5l18' 'kind_b1e' 'm4e21f'
 'hv3waz' 'hv3g1e' 'm5i20' 'm3i0p' 'm5f23k' 'm4b3b_2' 'p5q3dj' 'hv3p7_c'
 'cm4povco' 'm2h8b' 'p5j2h' 'cm4hhinc' 'f4b8f' 'm5g2d1' 'hv3l1' 'm5f23a'
 'hv3mwtkg' 'm4i8c2' 'm5f23e']


In [13]:
# Use Random forest to select features
# if test_var == 'gpa' or test_var == 'materialHardship': # regressor
fs_model_forest = RandomForestRegressor().fit(mapper.fit_transform(train_x), train_y.as_matrix())
# else:
#     fs_model_forest = RandomForestClassifier().fit(mapper.fit_transform(train_x), train_y.as_matrix())

fs_forest = SelectFromModel(fs_model_forest, prefit=True, threshold=0.005)
train_fs_forest_x = fs_forest.transform(train_x)
train_fs_forest_x = pd.DataFrame(data=train_fs_forest_x, index=indices, columns=colnames[fs_forest.get_support(True)])
train_fs_forest_x.head()

scores_forest = np.column_stack((colnames, fs_model_forest.feature_importances_))
scores_forest = scores_forest[fs_forest.get_support(True)]
scores_forest = np.core.records.fromarrays(scores_forest.transpose(), names='name, score', formats = 'S8, f8')
scores_forest = np.sort(scores_forest, order=['score'], kind='mergesort')
print("Random Forest # of features: {}".format(len(scores_forest)))
print(scores_forest['name'])

np.savetxt("{}_features_forest.csv".format(test_var), scores_forest['name'], fmt="%s")

# fig3 = plt.figure(3)
# dmy = fig3.clear()
# y_pos3 = np.arange(len(scores_forest))
# dmy = plt.title('Random Forest')
# dmy = plt.bar(y_pos3, scores_forest['score'], align='center')
# dmy = plt.xticks(y_pos3, scores_forest['name'], rotation='vertical')
# fig3

,m1citywt_rep6,m3i6d,f3citywt_rep7,m4f2c3,m4i14b,m4i23n,m4k7,m4k13,cm4hhinc,m5f14,m5f23e,m5f23g,m5f23k,hv3waz,hv4d4a,hv4k1a1,hv4l7,hv4wap,m4d7
challengeID,,,,,,,,,,,,,,,,,,,
266,0.089682,-0.071976,0.077432,-0.264114,-0.093647,0.435777,-0.080838,-0.390301,-0.249274,-0.174247,0.535373,0.528808,0.420744,-1.343664,-0.08925,-0.553978,-0.408980,0.511377,2.068034
2420,-0.413734,-0.071976,4.273507,0.042921,-0.093647,0.435777,-0.080838,-0.247014,0.109504,-0.174247,0.535373,0.528808,0.420744,0.095394,-0.08925,-0.553978,-0.408980,-2.105905,0.452459
1004,1.661215,-0.071976,1.376037,0.042921,-0.093647,0.435777,-0.080838,-0.436274,2.206003,-0.174247,0.535373,0.528808,0.420744,0.095394,-0.08925,-0.553978,-0.408980,-2.703631,-0.485340
1875,0.131493,-0.071976,-0.311349,-0.264114,-0.093647,-2.294750,-0.080838,-0.383461,-0.746030,-0.174247,0.535373,0.528808,0.420744,0.095394,-0.08925,-0.553978,1.941822,0.626893,-0.485340
104,-0.413734,-0.071976,-0.311349,-0.264114,-0.093647,-2.294750,-0.080838,-0.398119,-0.522543,-0.174247,0.535373,-1.891045,0.420744,0.095394,-0.08925,-0.553978,-0.408980,-1.776741,4.267485


Random Forest # of features: 19
['m4f2c3' 'm4d7' 'hv4l7' 'm4k7' 'hv4d4a' 'm4i14b' 'hv4wap' 'm5f14'
 'm1citywt' 'm4k13' 'hv3waz' 'f3citywt' 'hv4k1a1' 'cm4hhinc' 'm5f23g'
 'm3i6d' 'm4i23n' 'm5f23e' 'm5f23k']


In [14]:
# Use mutual info
fs_mutualinfo = SelectKBest(mutual_info_regression, k=50)
fs_mutualinfo.fit(mapper.fit_transform(train_x), train_y.as_matrix())
train_fs_mutualinfo_x = fs_mutualinfo.transform(train_x)
train_fs_mutualinfo_x = pd.DataFrame(data=train_fs_mutualinfo_x, index=indices, columns=colnames[fs_mutualinfo.get_support(True)])
train_fs_mutualinfo_x.head()

scores_mutualinfo = np.column_stack((colnames, fs_mutualinfo.scores_))
scores_mutualinfo = scores_mutualinfo[fs_mutualinfo.get_support(True)]
scores_mutualinfo = np.core.records.fromarrays(scores_mutualinfo.transpose(), names='name, score', formats = 'S8, f8')
scores_mutualinfo = np.sort(scores_mutualinfo, order=['score'], kind='mergesort')
print("MI # of features: {}".format(len(scores_mutualinfo)))
print(scores_mutualinfo['name'])

np.savetxt("{}_features_mutualinfo.csv".format(test_var), scores_mutualinfo['name'], fmt="%s")

# fig4 = plt.figure(4)
# dmy = fig4.clear()
# y_pos4 = np.arange(len(scores_mutualinfo))
# dmy = plt.title('Mutual Info')
# dmy = plt.bar(y_pos4, scores_mutualinfo['score'], align='center')
# dmy = plt.xticks(y_pos4, scores_mutualinfo['name'], rotation='vertical')

SelectKBest(k=50,
      score_func=<function mutual_info_regression at 0x7fce521d1848>)

,m1b12b,m1b12c,f1b1a,f1b26,f1e6d1,f2a7a1b,f2a8a,m3b32b,m3d3a_2,m3f2f3,...,cm5hhimp,cm5povca,hv3m36,hv4a1,hv4l47,ffcc_pof_e6,ffcc_centsurvey_c5a_13,ffcc_famsurvey_a12_a,ffcc_centobs_item_16,m4e23
challengeID,,,,,,,,,,,,,,,,,,,,,
266,0.635660,0.469897,-0.660108,-0.741634,-0.290441,-8.824754,0.082967,0.072883,-0.676189,-0.174247,...,-1.634440,0.875658,1.879423,0.837832,-0.304072,0.022271,-0.04611,0.015356,0.124191,0.294949
2420,0.635660,0.469897,-0.442686,-0.741634,-0.290441,-1.768200,0.082967,0.072883,-0.676189,-0.174247,...,-1.634440,-0.871129,1.879423,-0.582665,-0.304072,0.022271,-0.04611,0.015356,0.124191,0.294949
1004,0.635660,0.469897,-0.877529,-0.741634,-0.290441,0.722349,0.082967,0.072883,-0.676189,-0.174247,...,0.505026,-1.744522,-0.379719,-0.582665,-0.304072,0.022271,-0.04611,0.015356,0.124191,0.294949
1875,-2.262211,0.469897,-0.660108,-0.741634,-0.290441,0.307258,0.082967,0.072883,-0.676189,-0.174247,...,1.931336,-0.871129,-0.379719,0.837832,-0.304072,0.022271,-0.04611,0.015356,0.124191,0.294949
104,0.635660,0.469897,3.905752,-0.741634,3.443043,0.307258,0.082967,0.072883,-0.676189,-0.174247,...,0.505026,-0.871129,4.138565,-0.582665,-0.304072,-5.084337,-0.04611,0.015356,0.124191,0.294949


MI # of features: 50
['f5a3a1_1' 'c3citywt' 'f5a6g03_' 'ffcc_pof' 'hv3m36' 'f1b1a' 'f4a14a'
 'm1b12c' 'm3d3a_2' 'f5b6a1' 'p5i20a' 'f3c18' 'm4e23' 'm4i8c2' 'm5f23k'
 'cf4hhinc' 'f3c30a3' 'f3f2e3' 'm3i7a' 'f5j6c' 'f5a6g01_' 'f2a7a1b' 'hv4a1'
 'm3f2f3' 'm5j6c' 'f4j3' 'f1b26' 'm3b32b' 'm5i26a' 'f3k2_12' 'cm5povca'
 'p5m3c' 'm5e9_0' 'hv4l47' 'ffcc_cen' 'm1b12b' 'm5g18' 'k5b2a' 'f4c33'
 'm4l1' 'n5a11_1' 'f5i16a' 'ffcc_cen' 'f1e6d1' 'ffcc_fam' 'f2a8a' 'm3i3a'
 'cm5hhimp' 'f4c39' 'f5j1']


## Model Selection and Fitting

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

# For continuous regressors

# est- the regressor, params - params to CV, transform - to do any feature selection
regressors = [
    {
        'est': DecisionTreeRegressor(),
        'params': {
            "max_depth": [1000, None],
            "max_features": [None],
            "criterion": ["mse", "mae"]
        },
        'transform': [fs_decision, fs_lasso, fs_mutualinfo]
    },
    {
        'est': RandomForestRegressor(),
        'params': {"max_depth": [1000, None],
                  "max_features": [None],
                  "criterion": ["mse", "mae"]},
        'transform': [fs_forest, fs_lasso, fs_mutualinfo]
    },
    {
        'est': Lasso(),
        'params': {
            "alpha": [0.5, 1, 2, 4, 8],
            "fit_intercept": [True, False],
            "normalize": [True, False]
        },
        'transform': [fs_lasso, fs_mutualinfo]
    },
    {
        'est': LinearRegression(),
        'params': {
            "fit_intercept": [True, False],
            "normalize": [True, False]
        },
        'transform': [fs_lasso, fs_mutualinfo]
        
    },
    {
        'est': ElasticNet(),
        'params': {
            "alpha": [0.5, 1, 2, 4, 8],
            "l1_ratio": [0.1, 0.2, 0.4, 0.6, 0.8],
            "fit_intercept": [True, False],
            "normalize": [True, False],
            "positive": [True, False]
        },
        'transform': [fs_lasso, fs_mutualinfo]
    },
    {
        'est': LinearSVR(),
        'params': {
            "C": [1, 2, 4]
        },
        'transform': [fs_lasso, fs_mutualinfo]
        
    }
]

# classifiers for discrete vars
# classifiers = [
#     {
#         'est': MultinomialNB(),
#         'params': {
#             'alpha': [1, 0.1, 0.01, 0]
#                   },
#         'transform': [fs_lasso, fs_mutualinfo]
#     },
#     { 
#         'est': KNeighborsClassifier(),
#         'params': {
#             'n_neighbors': [2, 3, 4, 6, 10, 14, 18]
#         },
#         'transform': [fs_lasso, fs_mutualinfo]
#     },
#     {
#         'est': DecisionTreeClassifier(),
#         'params': {
#             'criterion': ['gini', 'entropy'],
#             'min_samples_split': [2, 4, 6, 8],
#             'max_depth': [1000, None],
#             'min_samples_leaf': [3, 6, 9, 12 ]
#         },
#         'transform': [fs_decision, fs_lasso, fs_mutualinfo]
#     },
#     {
#         'est': LinearSVC(),
#         'params': {
#             'C': [0.5, 1, 3, 6, 8]
#         },
#         'transform': [fs_lasso, fs_mutualinfo]
#     },
#     {
#         'est': RandomForestClassifier(),
#         'params': {
#             'criterion': ['gini', 'entropy'],
#             'max_depth': [1000, None],
#             'n_estimators': [5, 10, 20, 25]
#         },
#         'transform': [fs_forest, fs_lasso, fs_mutualinfo]
#     }
# ]

# if test_var == 'gpa' or test_var == 'materialHardship':
estimators = regressors
# else:
#     estimators = classifiers

# run grid search without feature selection
# keeping track of best classifier
best_mse = 100
best_clf = 0
best_transformer = 0
for est in estimators: 
    model = est['est']
    print("~~~ Fitting and Testing {} ~~~".format(type(model).__name__))
    grid_search = GridSearchCV(model, param_grid=est['params'])
    for transformer in est['transform']:
        "~Transforming with {}~".format(type(transformer).__name__)
        if transformer != None:
            train_fs_x = transformer.transform(train_x)
            test_fs_x = transformer.transform(test_x)
        else:
            train_fs_x = train_x
            test_fs_x = test_x

        grid_search.fit(train_fs_x, train_y)
        grid_search.get_params()

        est = grid_search.best_estimator_
        est.fit(train_fs_x, train_y)
        pred_y = est.predict(test_fs_x)

        mse = mean_squared_error(test_y, pred_y)
        "----MSE: {}----".format(mse)
        if (mse < best_mse):
            best_mse = mse
            best_est = est
            best_transformer = transformer


~~~ Fitting and Testing DecisionTreeRegressor ~~~


'~Transforming with SelectFromModel~'

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None], 'criterion': ['mse', 'mae'], 'max_depth': [1000, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

{'cv': None,
 'error_score': 'raise',
 'estimator': DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
 'estimator__criterion': 'mse',
 'estimator__max_depth': None,
 'estimator__max_features': None,
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_split': 1e-07,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__presort': False,
 'estimator__random_state': None,
 'estimator__splitter': 'best',
 'fit_params': {},
 'iid': True,
 'n_jobs': 1,
 'param_grid': {'criterion': ['mse', 'mae'],
  'max_depth': [1000, None],
  'max_features': [None]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': True,
 'scoring': None,
 'verbose': 0}

DecisionTreeRegressor(criterion='mse', max_depth=1000, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

...

'~Transforming with SelectKBest~'

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [1, 2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

{'cv': None,
 'error_score': 'raise',
 'estimator': LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
      intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
      random_state=None, tol=0.0001, verbose=0),
 'estimator__C': 1.0,
 'estimator__dual': True,
 'estimator__epsilon': 0.0,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1.0,
 'estimator__loss': 'epsilon_insensitive',
 'estimator__max_iter': 1000,
 'estimator__random_state': None,
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'fit_params': {},
 'iid': True,
 'n_jobs': 1,
 'param_grid': {'C': [1, 2, 4]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': True,
 'scoring': None,
 'verbose': 0}

LinearSVR(C=1, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

'----MSE: 0.0230048929026----'

In [19]:
# now predict for everything using best
print("Best estimator is {}".format(type(best_est).__name__))

full_trans_x = vt.transform(full_x)
full_trans_x = best_transformer.transform(full_trans_x)
pred_y = best_est.predict(full_trans_x)
len(pred_y)

# set the variable to be tested to output
if test_var == 'gpa':
    pred_gpa_y = pred_y
elif test_var == 'materialHardship':
    pred_material_y = pred_y
elif test_var == 'grit':
    pred_grit_y = pred_y
elif test_var == 'eviction':
    pred_evict_y= pred_y
elif test_var == 'layoff':
    pred_layoff_y = pred_y
elif test_var == 'jobTraining':
    pred_job_y = pred_y

prediction_np = np.column_stack([
    np.arange(1,n,dtype=int), pred_gpa_y, pred_grit_y, pred_material_y, pred_evict_y, pred_layoff_y, pred_job_y])

prediction = pd.DataFrame(prediction_np,
                          columns=["challengeID", "gpa", "grit", "materialHardship", "eviction", "layoff", "jobTraining"])
prediction.head()
prediction.tail()

prediction.to_csv("prediction_out.csv", index=False)

Best estimator is LinearRegression


4242

,challengeID,gpa,grit,materialHardship,eviction,layoff,jobTraining
0,1.0,2.866,3.427,0.028662,0.0596,0.209,0.334
1,2.0,2.866,3.427,0.177641,0.0596,0.209,0.334
2,3.0,2.866,3.427,0.028662,0.0596,0.209,0.334
3,4.0,2.866,3.427,0.128848,0.0596,0.209,0.334
4,5.0,2.866,3.427,0.028079,0.0596,0.209,0.334


,challengeID,gpa,grit,materialHardship,eviction,layoff,jobTraining
4237,4238.0,2.866,3.427,0.051861,0.0596,0.209,0.334
4238,4239.0,2.866,3.427,0.087648,0.0596,0.209,0.334
4239,4240.0,2.866,3.427,0.073632,0.0596,0.209,0.334
4240,4241.0,2.866,3.427,0.027178,0.0596,0.209,0.334
4241,4242.0,2.866,3.427,0.168805,0.0596,0.209,0.334
